<a href="https://colab.research.google.com/github/aetev/Learning-stuff-/blob/main/YF%20RL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install tf_agents[reverb]

In [24]:
import yfinance as yf
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy.lib.stride_tricks import sliding_window_view

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.regularizers import l2

from tf_agents.networks import network
from tf_agents.agents.categorical_dqn import categorical_dqn_agent
from tf_agents.environments import py_environment, tf_py_environment, utils
from tf_agents.specs import array_spec, tensor_spec
from tf_agents.trajectories import trajectory, time_step as ts
from tf_agents.replay_buffers.tf_uniform_replay_buffer import TFUniformReplayBuffer
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils import common

In [3]:
# Calculate the date 730 days ago
end_date = datetime.now()
start_date = end_date - timedelta(days=729)

# Download the data
data = yf.download('EURUSD=X', start=start_date, end=end_date, interval='1h')

# Print the data
print(data)

[*********************100%%**********************]  1 of 1 completed
                               Open      High       Low     Close  Adj Close  \
Datetime                                                                       
2021-10-18 07:00:00+01:00  1.157943  1.158346  1.157675  1.158212   1.158212   
2021-10-18 08:00:00+01:00  1.157943  1.159958  1.157943  1.159689   1.159689   
2021-10-18 09:00:00+01:00  1.159689  1.160093  1.158480  1.158480   1.158480   
2021-10-18 10:00:00+01:00  1.158614  1.159152  1.158078  1.158614   1.158614   
2021-10-18 11:00:00+01:00  1.158480  1.159958  1.158078  1.159555   1.159555   
...                             ...       ...       ...       ...        ...   
2023-10-17 01:00:00+01:00  1.055855  1.056189  1.055855  1.055855   1.055855   
2023-10-17 02:00:00+01:00  1.055743  1.055855  1.055298  1.055520   1.055520   
2023-10-17 03:00:00+01:00  1.055520  1.055520  1.054964  1.055298   1.055298   
2023-10-17 04:00:00+01:00  1.055298  1.055632  1.05

In [4]:
High = data['High'].to_numpy()
Close = data['Close'].to_numpy()
Low = data['Low'].to_numpy()
Std = np.std(np.diff(Close))
Close_Diff = np.diff(Close)/Std
High_Adj = (High[1:]-Close[1:])/Std
Low_Adj = (Close[1:]-Low[1:])/Std
# Get the sum of adjacent values
sum_array = Close_Diff[:-1] + Close_Diff[1:]
print(sum_array)

[ 0.21484273 -0.86061168  0.86042079 ... -0.44600625 -0.17838341
 -0.17828797]


In [5]:
stack = np.stack((Close_Diff, High_Adj, Low_Adj), axis=1)
print(stack)

[[ 1.18292349  0.21541539  1.39767077]
 [-0.96808076  1.29125156  0.        ]
 [ 0.10746909  0.43016267  0.42968546]
 ...
 [-0.17838341  0.17838341  0.26743195]
 [ 0.          0.2675274   0.17828797]
 [-0.17828797  0.17828797  0.        ]]


In [6]:
window_size = 50
window_view = np.lib.stride_tricks.sliding_window_view(stack, (window_size,),axis=0)

In [10]:
input = window_view[:-1]
target = Close_Diff[window_size:]

### <font color='red'><center>**-------------------Create Environment-------------------**</font>

In [ ]:
class TradingEnv(py_environment.PyEnvironment):

  def __init__(self):
    self._action_spec = array_spec.BoundedArraySpec(
        shape=(), dtype=np.int64, minimum=0, maximum=3, name='action')
    self._observation_spec = array_spec.BoundedArraySpec(
        shape=(12,), dtype=np.float32, minimum=-100,maximum=100, name='observation')


    self._state = [0] * 12
    self._episode_ended = False
    self._count = np.random.randint(10000,60000)
    self._end_ep = self._count+max_ep_len
    self._scaler = np.sum(np.abs(change[self._count:self._end_ep]))
    self._balance = 0
    self._prev_dff = 0
    self._returns = np.array([])
    self.trade_info = {
    'time': 0,
    'value': 0,
    'direction': 0,
    'max_val': 0,
    'min_val': 0,
    'max_time': 0,
    'min_time': 0
    }


  def action_spec(self):
    return self._action_spec

  def observation_spec(self):
    return self._observation_spec

  def get_balance(self):
    return self._balance

  def _reset(self):

    self._state = [0] * 12
    self._episode_ended = False
    self._count = np.random.randint(10000,60000)
    self._end_ep = self._count+max_ep_len
    self._scaler = np.sum(np.abs(change[self._count:self._end_ep]))
    self._returns = np.array([])
    self._balance = 0
    self._prev_diff = 0
    self.trade_info = {
    'time': 0,
    'value': 0,
    'direction': 0,
    'max_val': 0,
    'min_val': 0,
    'max_time': 0,
    'min_time': 0
    }

    return ts.restart(np.array(self._state, dtype=np.float32))

  def _step(self, action):

    if self._episode_ended:
      # The last action ended the episode. Ignore the current action and start
      # a new episode.
      return self.reset()


    mult = 10
    diff_hold = DIFF_hold[self._count]
    profit = 0
    reward = 0
    cur_change = change[self._count+1]/mult

    cur_direction = self.trade_info['direction']

    if action == 0:
      profit = self.trade_info['value']-.005
      reward = profit
      self.trade_info['value'] = cur_change
      self.trade_info['direction'] = 1
      self.trade_info['time'] = 0
      self.trade_info['max_val'] = cur_change
      self.trade_info['min_val'] = cur_change
      self.trade_info['max_time'] = 0
      self.trade_info['min_time'] = 0
    elif action == 1:
      profit = self.trade_info['value']-.005
      reward = profit
      self.trade_info['value'] = -cur_change
      self.trade_info['direction'] = -1
      self.trade_info['time'] = 0
      self.trade_info['max_val'] = -cur_change
      self.trade_info['min_val'] = -cur_change
      self.trade_info['max_time'] = 0
      self.trade_info['min_time'] = 0
      #reward = profit if profit > 0 else profit/2
    elif action == 2:
      profit = self.trade_info['value']
      reward = profit
      self.trade_info['value'] = 0
      self.trade_info['direction'] = 0
      self.trade_info['time'] = 0 if cur_direction != 0 else self.trade_info['time'] + .01
      self.trade_info['max_val'] = 0
      self.trade_info['min_val'] = 0
      self.trade_info['max_time'] = 0
      self.trade_info['min_time'] = 0
    else:
      self.trade_info['value'] += cur_change*cur_direction
      self.trade_info['time'] += .01
      self.trade_info['max_val'] = max(self.trade_info['value'],self.trade_info['max_val'])
      self.trade_info['min_val'] = min(self.trade_info['value'],self.trade_info['min_val'])
      self.trade_info['max_time'] = 0 if self.trade_info['max_val']==self.trade_info['value'] else self.trade_info['max_time']+.01
      self.trade_info['min_time'] = 0 if self.trade_info['min_val']==self.trade_info['value'] else self.trade_info['min_time']+.01



    self._balance += profit
    if profit != 0:
        self._returns = np.append(self._returns, profit)

    info_list = [
        self.trade_info['value'],
        self.trade_info['direction'],
        self.trade_info['time'],
        self.trade_info['max_val'],
        self.trade_info['min_val'],
        self.trade_info['max_time'],
        self.trade_info['min_time']
    ]


    #update state
    self._state = np.append(INDICATORS[self._count],info_list)

    if self._count >= self._end_ep:
      self._episode_ended = True
      self._returns.append(self.trade_info['value'])

    self._count += 1

    if self._episode_ended:
      #self._returns = [x for x in self._returns if x >= 0]
      #reward = np.sum(self._returns)/self._scaler if np.sum(self._returns) != 0 else -1
      #reward = np.mean(self._returns) if np.mean(self._returns) != 0 else -10
      reward = SortinoRatio(self._returns,0) if np.sum(self._returns) != 0 else -10
      return ts.termination(np.array(self._state, dtype=np.float32), reward=reward)
    else:
      return ts.transition(
          np.array(self._state, dtype=np.float32), reward=reward, discount=.999)

In [ ]:
env = TradingEnv()
utils.validate_py_environment(env, episodes=2)

print('Observation Spec:')
print(train_env.time_step_spec().observation)

print('Reward Spec:')
print(train_env.time_step_spec().reward)

print('Action Spec:')
print(train_env.action_spec())

train_env = tf_py_environment.TFPyEnvironment(env)
evaluation_env = tf_py_environment.TFPyEnvironment(env)

### <font color='blue'><center>**-------------------Create Network-------------------**</font>

In [9]:
class CustomQNetwork(network.Network):
    def __init__(self, input_tensor_spec, action_spec, name='CustomQNetwork'):
        super(CustomQNetwork, self).__init__(input_tensor_spec=input_tensor_spec,
                                             state_spec=(),
                                             name=name)
        # Define your custom layers and architecture here
        self.dense1 = layers.Dense(100,activation='gelu')
        self.drop1 = layers.Dropout(.2)
        self.dense2 = layers.Dense(100,activation='gelu')
        self.drop2 = layers.Dropout(.2)
        self.dense2 = layers.Dense(100,activation='gelu')
        self.drop2 = layers.Dropout(.2)
        self.q_values_layer = tf.keras.layers.Dense(action_spec.maximum - action_spec.minimum + 1,
                                                    activation=None)

    def call(self, observation, step_type=None, network_state=(), training=False):
        # Pass the observation through your custom layers
        x = self.dense1(observation)
        x = self.drop1(x)
        x = self.dense2(x)
        x = self.drop2(x)
        q_values = self.q_values_layer(x)
        return q_values, network_state

In [ ]:
class CategoricalQNetwork(network.Network):
    def __init__(self, input_tensor_spec, action_spec, num_atoms=51, preprocessing_layers=None,
                 preprocessing_combiner=None, conv_layer_params=None, fc_layer_params=None,
                 activation_fn=tf.nn.relu, name='CategoricalQNetwork'):
        super(CategoricalQNetwork, self).__init__(input_tensor_spec=input_tensor_spec,
                                                  state_spec=(),
                                                  name=name)
        # Check if action_spec is a BoundedTensorSpec
        if not isinstance(action_spec, tensor_spec.BoundedTensorSpec):
            raise TypeError('action_spec must be a BoundedTensorSpec. Got: %s' % (action_spec,))

        self._num_actions = action_spec.maximum - action_spec.minimum + 1
        self._num_atoms = num_atoms

        # Create a BoundedTensorSpec for the q_network action
        q_network_action_spec = tensor_spec.BoundedTensorSpec((), tf.int32,
                                                              minimum=0,
                                                              maximum=self._num_actions * num_atoms - 1)

        # Create the q_network
        self._q_network = CustomQNetwork(input_tensor_spec=input_tensor_spec,
                                             action_spec=q_network_action_spec,
                                             )
    def call(self, observation, step_type=None, network_state=(), training=False):
        logits, network_state = self._q_network(observation, step_type, network_state, training=training)
        logits = tf.reshape(logits, [-1, self._num_actions, self._num_atoms])
        return logits, network_state
    @property
    def num_atoms(self):
        return self._num_atoms

In [20]:
observation_spec = train_env.observation_spec()
action_spec = train_env.action_spec()
num_atoms = 51
custom_network = CategoricalQNetwork(observation_spec,action_spec, num_atoms)

NameError: ignored

In [ ]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=.01)

train_step_counter = tf.Variable(0)

C51_agent = categorical_dqn_agent.CategoricalDqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    categorical_q_network=custom_network,
    optimizer=optimizer,
    min_q_value=-5,
    max_q_value=5,
    n_step_update=2,
    td_errors_loss_fn=common.element_wise_squared_loss,
    gamma=1,
    train_step_counter=train_step_counter)
C51_agent.initialize()

### <font color='green'><center>**-------------------Create Replay Buffer And Training Loop-------------------**</font>

In [25]:
def get_average_reward(environment, policy, episodes=10):

    total_reward = 0.0

    for _ in range(episodes):
        time_step = environment.reset()
        episode_reward = 0.0

    while not time_step.is_last():
        action_step = policy.action(time_step)
        time_step = environment.step(action_step.action)
        episode_reward += time_step.reward

    total_reward += episode_reward
    avg_reward = total_reward / episodes

    return avg_reward.numpy()[0]

In [26]:
class ExperienceReplay(object):
    def __init__(self, agent, enviroment):
        self._replay_buffer = TFUniformReplayBuffer(
            data_spec=agent.collect_data_spec,
            batch_size=enviroment.batch_size,
            max_length=10000)

        self._random_policy = RandomTFPolicy(train_env.time_step_spec(),
                                                enviroment.action_spec())

        self._fill_buffer(train_env, self._random_policy, steps=100)

        self.dataset = self._replay_buffer.as_dataset(
            num_parallel_calls=tf.data.AUTOTUNE,
            sample_batch_size=BATCH_SIZE,
            num_steps=2).prefetch(tf.data.AUTOTUNE)

        self.iterator = iter(self.dataset)



    def _fill_buffer(self, enviroment, policy, steps):
        for _ in range(steps):
            self.timestamp_data(enviroment, policy)

    def timestamp_data(self, environment, policy):
        time_step = environment.current_time_step()
        action_step = policy.action(time_step)
        next_time_step = environment.step(action_step.action)
        timestamp_trajectory = trajectory.from_transition(time_step, action_step, next_time_step)

        self._replay_buffer.add_batch(timestamp_trajectory)

In [ ]:
def train(agent):
    experience_replay = ExperienceReplay(agent, train_env)

    agent.train_step_counter.assign(0)

    avg_reward = get_average_reward(evaluation_env, agent.policy, EVAL_EPISODES)
    rewards = [avg_reward]

    for _ in range(NUMBER_EPOSODES):

        for _ in range(COLLECTION_STEPS):
            experience_replay.timestamp_data(train_env, agent.collect_policy)

        experience, info = next(experience_replay.iterator)
        train_loss = agent.train(experience).loss

        if agent.train_step_counter.numpy() % EVAL_INTERVAL == 0:
            avg_reward = get_average_reward(evaluation_env, agent.policy, EVAL_EPISODES)
            print('Episode {0} - Average reward = {1}, Loss = {2}.'.format(
					agent.train_step_counter.numpy(), avg_reward, train_loss))
            rewards.append(avg_reward)

    return rewards


print("**********************************")
print("Training DDQN")
print("**********************************")
C51_reward = train(C51_agent)